# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [7]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    # Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

        self.system_prompt = "You are an assistant that analyzes the contents of a website \
                    and provides a short summary, ignoring text that might be navigation related. \
                    Respond in markdown."

    def user_prompt_for(self):
        user_prompt = f"You are looking at a website titled {self.title}"
        user_prompt += "\nThe contents of this website is as follows; \
                        please provide a short summary of this website in markdown. \
                        If it includes news or announcements, then summarize these too.\n\n"
        user_prompt += self.text
        return user_prompt

    def messages_for(self):
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt_for()}
        ]

    def summarize(self, llmHosturl='http://localhost:11434/v1', api_key='ollama', MODEL="llama3.2", tryOpenS=True):
        if tryOpenS:
            ollama_via_openai = OpenAI(base_url=llmHosturl, api_key=api_key)
            response = ollama_via_openai.chat.completions.create(
                model=MODEL,
                messages=self.messages_for()
            )
        else:
            response = openai.chat.completions.create(
                model = "gpt-4o-mini",
                messages = self.messages_for()
            )
        return response.choices[0].message.content

    def display_summary(self, summary):
        display(Markdown(summary))

In [8]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")

In [9]:
summarize_website = ed.summarize()

In [10]:
ed.display_summary(summarize_website)

**Summary**
================

Ed Donner's website is a personal blog that showcases his interests in artificial intelligence, programming, and music production. He is the co-founder and CTO of Nebula.io, an AI startup that applies machine learning to help people discover their potential. The website features links to donner's social media profiles and newsletter subscription.

**News/Announcements**
----------------------

*   **December 21, 2024:** A welcome message for SuperDataScientists.
*   **November 13, 2024:** "Mastering AI and LLM Engineering – Resources" - a blog post featuring resources for mastering AI and LLM engineering.
*   **October 16, 2024:** "From Software Engineer to AI Data Scientist – resources" - a blog post featuring resources for transitioning from software engineer to AI data scientist.
*   **August 6, 2024:** "Outsmart LLM Arena – a battle of diplomacy and deviousness" - an introduction to the Outsmart LLM Arena, where LLMs are pitted against each other in a battle of diplomacy and deviousness.

**Other Features**
-------------------

[An arena that pits LLMs against each other](#outsmart) 
### outsmart

An arena that pits LLMs against each other in a battle of diplomacy and deviousness

In [11]:
ant = Website("https://anthropic.com")

In [12]:
summarize_ant = ant.summarize()

In [13]:
ant.display_summary(summarize_ant)

# Website Summary

## Overview

Anthropic is an AI safety and research company based in San Francisco, focusing on creating reliable and beneficial AI systems.

### Research and Products

Anthropic develops cutting-edge AI products, including Claude, a family of intelligent AI models that prioritize safety. Recent developments include the introduction of Claude 3.5 Sonnet and Haiku, advancements in alignment and constitutional AI harmlessness.

### News and Announcements

* **Introducing new Claude updates**: A new version of Claude has been released to improve performance.
* **Core views on AI safety**: Published on March 8, 2023, providing guidance on when, why, what, and how to focus on AI safety.
* **Constitutional AI: Harmlessness from AI Feedback**: Announced on December 15, 2022, investigating harmlessness in AI feedback.

## Company Information

Anthropic's interdisciplinary team comprises experts from ML, physics, policy, and product. The company generates research and creates beneficial AI systems, with open roles available for collaboration.